In [1]:
import ee 
import folium
from datetime import datetime as dt
from IPython.display import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from google.cloud import storage
import time
import numpy as np
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers
from google.colab import auth

In [2]:
auth.authenticate_user()

In [3]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-6nmIvcR9FYQBW6AeW-RgGUOqGZPIZqvyhnp52pVqxk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7cG0VnGRp25zB9iGF9VP1ChgHlAF7bBY6upff6HSMAura3yBWBHmY

Successfully saved authorization token.


In [4]:
ee.Initialize()

## Global variables

In [5]:
BUCKET = 'forest_guard_bucket'  # ⚠️ replace with your BUCKET NAME
FOLDER = 'test_feature_eng_laurent'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

OPTICAL_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
THERMAL_BANDS = ['B10', 'B11']
BANDS = OPTICAL_BANDS + THERMAL_BANDS
RESPONSE = 'fnf'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
   

## Data

In [6]:
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function from GEE
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(OPTICAL_BANDS).gt(0).And(
          image.select(OPTICAL_BANDS).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(OPTICAL_BANDS).divide(10000).addBands(
          image.select(THERMAL_BANDS).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

# The image input data is a cloud-masked median composite.
image = l8sr.filterDate('2015-01-01', '2017-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[48.9, 2.5])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
    opacity = 0.7
  ).add_to(map)

mapid = image.getMapId({'bands': ['B11'], 'min': 0, 'max': 0.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='thermal',
    opacity = 0.7
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Target

In [7]:
jaxa = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF').filterDate('2017-01-01', '2017-12-31').median()

In [8]:



palette = ['006400',
           'FEFF99',
          '0000FF']
mapid = jaxa.getMapId({'bands': ['fnf'],
                       'min': 1, 
                       'max': 3, 
                      'palette':palette
                      })
map = folium.Map(location=[48.9, 2.5])


folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='jaxa fnf',
    color=palette,
    opacity = 1
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Stacking X and y

In [15]:
featureStack = ee.Image.cat([
  image.select(BANDS),
  jaxa.select(RESPONSE)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

## Areas of interest

In [16]:
# create our own areas of interests
# training features collection
rect_train = ee.Geometry.Rectangle([3.327248985602229, 46.600827791084875, 4.689553673102229,47.70649093701327])
trainingPolys = ee.FeatureCollection([rect_train])
# eval features collections
rect_eval = ee.Geometry.Rectangle([-0.7016469211726672,46.88962161312492, 0.9353159694523328,47.64998631126759])
evalPolys= ee.FeatureCollection([rect_eval])

In [17]:
###################################################
###   DEFINE OUR COLLECTSIONS OF AREAS OF INTEREST
################################################
# trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
# evalPolys = ee.FeatureCollection('projects/google/DemoEvalGeometries')

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[47., 0.], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [18]:
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

## Export data to Google Cloud Storage in TFRecord format

In [19]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
        sample = arrays.sample(
          region = ee.Feature(trainingPolysList.get(g)).geometry(), 
          scale = 30,
          numPixels = N / n, # Size of the shard.
          seed = i,
          tileScale = 8
        )
        geomSample = geomSample.merge(sample)

    desc = TRAINING_BASE + '_g' + str(g)

    
    task = ee.batch.Export.table.toCloudStorage(
                                                collection = geomSample,
                                                description = desc,
                                                bucket = BUCKET,
                                                fileNamePrefix = FOLDER + '/' + desc,
                                                fileFormat = 'TFRecord',
                                                selectors = BANDS + [RESPONSE], 
                                                )
    task.start()
        
# Monitor task progress
# Code Extracted here:
# https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb
# import time 
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)
	
# print('Done!')

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
        sample = arrays.sample(
                              region = ee.Feature(evalPolysList.get(g)).geometry(), 
                              scale = 30,
                              numPixels = N / n,
                              seed = i,
                              tileScale = 8
                            )
        geomSample = geomSample.merge(sample)

desc = EVAL_BASE + '_g' + str(g)
task = ee.batch.Export.table.toCloudStorage(
                                        collection = geomSample,
                                        description = desc,
                                        bucket = BUCKET,
                                        fileNamePrefix = FOLDER + '/' + desc,
                                        fileFormat = 'TFRecord',
                                        selectors = BANDS + [RESPONSE]
                                        )
task.start()
task.status()
# Monitor task progress
# Code Extracted here:
# https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)
task.status()

Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id: QMDSKJWSN2P5UBMK4AAPOUNT).
Polling for task (id

{'attempt': 1,
 'creation_timestamp_ms': 1622128135345,
 'description': 'eval_patches_g0',
 'destination_uris': ['https://console.developers.google.com/storage/browser/forest_guard_bucket/test_feature_eng_laurent/'],
 'id': 'QMDSKJWSN2P5UBMK4AAPOUNT',
 'name': 'projects/earthengine-legacy/operations/QMDSKJWSN2P5UBMK4AAPOUNT',
 'start_timestamp_ms': 1622128180899,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1622128405770}

## Parse TFRecords and get back tensors

In [20]:
def parse_tfrecord(example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
    example_proto: a serialized Example.
    Returns:
    A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
    inputs: A dictionary of tensors, keyed by feature name.
    Returns:
    A tuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
    A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(to_tuple, num_parallel_calls=5)
    return dataset

In [21]:
def get_training_dataset():
    """Get the preprocessed training dataset
    Returns: 
    A tf.data.Dataset of training data.
    """
    glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    return dataset

training = get_training_dataset()

# print(iter(training.take(1)).next())

In [22]:
def get_eval_dataset():
    """Get the preprocessed evaluation dataset
    Returns: 
    A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

evaluation = get_eval_dataset()

In [23]:
print(iter(training.take(1)).next())

(<tf.Tensor: shape=(16, 256, 256, 9), dtype=float32, numpy=
array([[[[0.0291    , 0.0339    , 0.0584    , ..., 0.1115    ,
          0.1735    , 0.1555    ],
         [0.03195   , 0.0369    , 0.07195   , ..., 0.1214    ,
          0.172     , 0.1545    ],
         [0.0304    , 0.03555   , 0.06875   , ..., 0.1074    ,
          0.1725    , 0.155     ],
         ...,
         [0.03295   , 0.0417    , 0.0761    , ..., 0.1384    ,
          0.177     , 0.16      ],
         [0.0453    , 0.0567    , 0.0868    , ..., 0.183     ,
          0.1805    , 0.1545    ],
         [0.04315   , 0.05115   , 0.0819    , ..., 0.1705    ,
          0.1775    , 0.1545    ]],

        [[0.0247    , 0.0296    , 0.0502    , ..., 0.096     ,
          0.1745    , 0.1535    ],
         [0.03055   , 0.0356    , 0.06845   , ..., 0.1234    ,
          0.172     , 0.155     ],
         [0.0301    , 0.03495   , 0.0682    , ..., 0.1103    ,
          0.175     , 0.159     ],
         ...,
         [0.02835   , 0.0347

In [ ]:
evaluation, training

(<RepeatDataset shapes: ((None, 256, 256, 9), (None, 256, 256, 1)), types: (tf.float32, tf.float32)>,
 <RepeatDataset shapes: ((None, 256, 256, 9), (None, 256, 256, 1)), types: (tf.float32, tf.float32)>)

In [24]:
def target_simplification(x,y):
  y_simplified = tf.where(y>1., 0., y)
  return (x,y_simplified)

In [25]:
evaluation_s = evaluation.map(target_simplification).repeat()
training_s = training.map(target_simplification).repeat()

In [ ]:
type(evaluation_s) == type(evaluation)

True

In [26]:
iter(evaluation_s.take(1)).next()

(<tf.Tensor: shape=(1, 256, 256, 9), dtype=float32, numpy=
 array([[[[0.0299 , 0.0388 , 0.0691 , ..., 0.1354 , 0.1855 , 0.1675 ],
          [0.0299 , 0.0381 , 0.0692 , ..., 0.1327 , 0.1865 , 0.1675 ],
          [0.0283 , 0.0348 , 0.0624 , ..., 0.1113 , 0.1885 , 0.1695 ],
          ...,
          [0.0194 , 0.02575, 0.0452 , ..., 0.0711 , 0.166  , 0.1485 ],
          [0.0179 , 0.0241 , 0.04145, ..., 0.06995, 0.1675 , 0.15   ],
          [0.01835, 0.02345, 0.039  , ..., 0.0654 , 0.1565 , 0.138  ]],
 
         [[0.0302 , 0.0385 , 0.0675 , ..., 0.13355, 0.1925 , 0.1725 ],
          [0.0319 , 0.03995, 0.0682 , ..., 0.13675, 0.1935 , 0.1735 ],
          [0.0327 , 0.0392 , 0.07045, ..., 0.133  , 0.194  , 0.173  ],
          ...,
          [0.01905, 0.02435, 0.04325, ..., 0.06935, 0.165  , 0.1475 ],
          [0.01965, 0.02375, 0.04745, ..., 0.07315, 0.1665 , 0.149  ],
          [0.0187 , 0.0254 , 0.0455 , ..., 0.0737 , 0.1625 , 0.1395 ]],
 
         [[0.03125, 0.03835, 0.06695, ..., 0.13695, 0

In [27]:
# Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
MODEL_DIR = 'gs://ee-docs-demos/fcnn-demo/trainer/model'
model_pretrained = tf.keras.models.load_model(MODEL_DIR)
model_pretrained.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 2624        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [30]:
from tensorflow.keras.metrics import MeanIoU
iou = MeanIoU(num_classes=2)

In [31]:
model_pretrained.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[iou, "mae", "accuracy"])

In [32]:
history = model_pretrained.fit(
   x=training_s, 
    epochs=1, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation_s,
    validation_steps=EVAL_SIZE 
)

   8/1000 [..............................] - ETA: 10:53:18 - loss: 0.3516 - mean_io_u_1: 0.3204

KeyboardInterrupt: ignored

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['mse'], label='train mse'  + exp_name)
    ax2.plot(history.history['val_mse'], label='val mse'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)

NameError: ignored

In [ ]:
MODEL_SAVE = 'gs://' + BUCKET + '/' + 'model_trained'
model_pretrained.save(MODEL_SAVE)

INFO:tensorflow:Assets written to: gs://forest_guard_bucket/model_trained/assets


## Prediction

In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}